In [1]:
import pandas as pd
import numpy as np
import os
import shapely
from shapely.geometry import Point, Polygon
import geopandas as gpd

print("done")


done


In [2]:
airqual_may=pd.read_csv("D:\\Data projects\\Air pollution Madrid\\data\\air data\\Calidad aire 2024\\pollution\\may_mo24.csv", sep=";")
airqual_abr=pd.read_csv("D:\\Data projects\\Air pollution Madrid\\data\\air data\\Calidad aire 2024\\pollution\\abr_mo24.csv", sep=";")

In [3]:
# Function to extract the numbers before the first underscore
def extract_numbers(value):
    return value.split('_')[0]

# List of files to process with full paths
files = [
    r'D:\\Data projects\\Air pollution Madrid\\data\\air data\\Calidad aire 2024\\pollution\\may_mo24.csv',
    r'D:\\Data projects\\Air pollution Madrid\\data\\air data\\Calidad aire 2024\\pollution\\abr_mo24.csv'
]

for file in files:
    # Read the CSV file
    df = pd.read_csv(file, sep=";")
    
    # Check if the column 'PUNTO_MUESTREO' exists in the DataFrame
    if 'PUNTO_MUESTREO' in df.columns:
        # Create the new column and fill it with the extracted values
        df['ID_EST'] = df['PUNTO_MUESTREO'].apply(extract_numbers)
        
        # Reorder columns to make the new column the first one
        columns_order = ['ID_EST'] + [col for col in df.columns if col != 'ID_EST']
        df = df[columns_order]
        #drop unnecessary cols
        df.drop(df.columns[[1, 2, 3]], axis=1, inplace=True)
        # Find columns that start with 'V'
        columns_with_v = [col for col in df.columns if col.startswith('V')]

        # Keep rows where any of these columns have the value 'V'
        # Variables not assigned a V correspond to not valid measurements
        for col in columns_with_v:
            df = df[df[col] == 'V']

        # Drop all validation columns 
        df = df.loc[:, ~df.columns.str.startswith('V')]
        # Reshape the DataFrame to have hourly data in rows
        df_melted = df.melt(id_vars=['ID_EST', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES', 'DIA'],
                            var_name='HOUR', value_name='VALUE')
        
        # Extract the hour from the 'HOUR' column
        df_melted['HOUR'] = df_melted['HOUR'].str.extract('(\d+)', expand=False).astype(int)
        
        # Sort the DataFrame by ID_EST, MAGNITUD, and timestamp
        df_melted = df_melted.sort_values(by=['ID_EST', 'MAGNITUD', 'ANO', 'MES', 'DIA', 'HOUR'])
        # Save the modified DataFrame to a new CSV file
        output_file = file.replace('.csv', '_modified.csv')
        df_melted.to_csv(output_file, index=False)
        

In [4]:
#concatenate the modified files 
may = pd.read_csv('D:\\Data projects\\Air pollution Madrid\\data\\air data\\Calidad aire 2024\\pollution\\may_mo24_modified.csv',sep=",")
apr = pd.read_csv('D:\\Data projects\\Air pollution Madrid\\data\\air data\\Calidad aire 2024\\pollution\\abr_mo24_modified.csv',sep=",")
air_data = pd.concat([apr, may], ignore_index=True)


In [5]:
# Now we merge the air_data with the coordinates of each station
# Load the CSV file
file_path = "D:\\Data projects\\Air pollution Madrid\\data\\locations\\air_locations_modified.csv"
locations_df = pd.read_csv(file_path, sep=",")


# Convert the ID_EST column to int to match the ID column in locations_df
air_data['ID_EST'] = air_data['ID_EST'].astype(int)

# Ensure the ID column in locations_df is also of type int (it usually should be)
locations_df['ID'] = locations_df['ID'].astype(int)

# Merge the DataFrames on the corrected ID columns
merged_df = pd.merge(air_data, locations_df, left_on='ID_EST', right_on='ID', how='left')

# Specify the directory and filename to save the combined DataFrame
output_directory = r"D:\\Data projects\\Air pollution Madrid\\data\\air data\\Calidad aire 2024\\pollution"
output_filename = 'air_geolocated.csv'
output_file = os.path.join(output_directory, output_filename)

# Save the combined DataFrame to a new CSV file
merged_df.to_csv(output_file, index=False)

print("This is the resulting dataset")
merged_df

This is the resulting dataset


,ID_EST,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,HOUR,VALUE,ID,LATITUD,LONGITUD
0,28079004,1,28079004_1_38,2024,4,1,1,1.0,28079004,40.423882,-3.712257
1,28079004,1,28079004_1_38,2024,4,1,2,1.0,28079004,40.423882,-3.712257
2,28079004,1,28079004_1_38,2024,4,1,3,1.0,28079004,40.423882,-3.712257
3,28079004,1,28079004_1_38,2024,4,1,4,1.0,28079004,40.423882,-3.712257
4,28079004,1,28079004_1_38,2024,4,1,5,1.0,28079004,40.423882,-3.712257
...,...,...,...,...,...,...,...,...,...,...,...
161395,28079060,14,28079060_14_6,2024,5,31,20,105.0,28079060,40.500548,-3.689731
161396,28079060,14,28079060_14_6,2024,5,31,21,100.0,28079060,40.500548,-3.689731
161397,28079060,14,28079060_14_6,2024,5,31,22,95.0,28079060,40.500548,-3.689731
161398,28079060,14,28079060_14_6,2024,5,31,23,90.0,28079060,40.500548,-3.689731
